In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Amazon_Unlocked_Mobile.csv')

In [ ]:
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [ ]:
df1 = df.drop(['Product Name', 'Brand Name', 'Price', 'Review Votes'], axis=1)
df1['label'] = df1['Rating'].apply(lambda c: 1 if c >3 else (0 if c<3 else 'neut'))
df1 = df1.drop(['Rating'], axis=1)
df1 = df1[df1['label'] != 'neut']
df1 = df1.replace(np.nan, '', regex=True)

In [ ]:
def get_top_data(top_n):
    top_data_df_positive = df1[df1['label'] == 1].head(top_n)
    top_data_df_negative = df1[df1['label'] == 0].head(top_n)
    top_data_df_small = pd.concat([top_data_df_positive, top_data_df_negative])
    return top_data_df_small

In [ ]:
# Function call to get the top 30000 from each sentiment
df1 = get_top_data(top_n=30000)

# After selecting top few samples of each sentiment
print("After segregating and taking equal number of rows for each sentiment:")
print(df1['label'].value_counts())
df1.head(10)

After segregating and taking equal number of rows for each sentiment:
1    30000
0    30000
Name: label, dtype: int64


,Reviews,label
0,I feel so LUCKY to have found this used (phone...,1
1,"nice phone, nice up grade from my pantach revu...",1
2,Very pleased,1
3,It works good but it goes slow sometimes but i...,1
4,Great phone to replace my lost phone. The only...,1
8,I originally was using the Samsung S2 Galaxy f...,1
11,This is a great product it came after two days...,1
12,These guys are the best! I had a little situat...,1
14,Ordered this phone as a replacement for the sa...,1
16,I was able to get the phone I previously owned...,1


In [ ]:
max_len_text = 2000
train_set = 45000

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df1['Reviews'])
vector = tokenizer.texts_to_sequences(df1['Reviews'])

In [ ]:
train_data = np.array(vector)[:train_set]
train_label = (np.array(df1['label'])[:train_set])
test_data = np.array(vector)[train_set:]
test_label = (np.array(df1['label'])[train_set:])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
len(test_data)

15000

In [ ]:
padded_train = pad_sequences(train_data, maxlen=max_len_text)
padded_test = pad_sequences(test_data, maxlen=max_len_text)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(13000, 16, input_length=max_len_text))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.2)))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2000, 16)          208000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               41472     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 249,601
Trainable params: 249,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(padded_train, train_label, epochs=4, validation_data=(padded_test, test_label))

Epoch 1/4
1407/1407 [==============================] - 189s 128ms/step - loss: 0.2063 - accuracy: 0.9187 - val_loss: 0.6891 - val_accuracy: 0.7583
Epoch 2/4
1407/1407 [==============================] - 182s 129ms/step - loss: 0.0968 - accuracy: 0.9701 - val_loss: 0.4288 - val_accuracy: 0.8454
Epoch 3/4
1407/1407 [==============================] - 185s 131ms/step - loss: 0.0802 - accuracy: 0.9751 - val_loss: 0.6512 - val_accuracy: 0.8123
Epoch 4/4
1407/1407 [==============================] - 186s 132ms/step - loss: 0.0611 - accuracy: 0.9824 - val_loss: 0.4434 - val_accuracy: 0.8531
